# From Social Science to Data Science: Preparing a Stack Exchange DataFrame

**Author**: Bernie Hogan, bernie.hogan@oii.ox.ac.uk

**Last Modification**: October 17, 2023

Run this cell below for a one-click solution to get from any Stack Exchange archive to a raw XML file and DataFrame pickle that you can use with Chapters 10-13 of Hogan's _From Social Science to Data Science_. 

See bottom of file for change log. 

## To use

Simply run the cell. The rest should be self-explanatory from the widgets (i.e. buttons) it produces. Please note that all required packages should have already been installed with anaconda, except possibly `py7zr` which should download and install automatically from `pip`. 

## Features

This is script is very advanced compared to what you have seen in the book, but it shouldn't be completely intimidating. Compared to the code in Chapter 10, here are some interesting features that you might want to check out: 

- I use `ipywidgets` in order to create buttons instead of just running code. 
- I use `tqdm` to create a progress bar for the file. 
- I update the status of the processing during the work.
- I use `ElementTree` instead of `json_normalise()`. It's more fussy, but it's faster.
- I refactored the `cleanBody` and `cleanTags` so that it is only one call to BeautifulSoup.
- I use `multiprocessing` in order to make the processing of text work a lot faster on modern multi-core processors. 
- I have options so that you can keep the original 7zip archive if you wish. 
- The methods should all have docstrings and type hinting. 
- ChatGPT gave lots of useful pointers and tips, but no large code snippets.

## Future extensions 

This script hardcodes for several things which might be made more general. 
- The original archive at "https://archive.org/download/stackexchange"
- The default download directory (in a sibling folder to the parent of this file under 'data')

It would be useful in the future to:
- Ensure compatibility when storing data on Google Drive for Google Colab.

In [ ]:
import pandas as pd 
import ipywidgets as iw
from tqdm.notebook import tqdm
from IPython.display import display, Markdown
from pathlib import Path
import requests
import warnings
import os
import time
import bs4 
import xml.etree.ElementTree as ET
import multiprocessing

# This might not be installed. It shouldn't cause trouble to live install
try: 
    import py7zr
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install py7zr
    import py7zr


URL = "https://archive.org/download/stackexchange"
DOWNLOAD_FOLDER = Path().cwd().parent / "data"

def download_table(url = "https://archive.org/download/stackexchange"):
    """
    Downloads URL and parses for tables, and returns first table, which is a directory listing of all Stack Exchanges. 

    Parameters: 
    url (str): A path to the directory listing.

    Returns: 
    pd.DataFrame: Table as unformatted DataFrame.
    """

    result = pd.read_html(url)
    return result[0]


def process_archive_table(df: pd.DataFrame) -> pd.DataFrame: 
    """
    Processes the text from an Internet Archive listing of Stack Exchanges.

    Returns: 
    pd.DataFrame: A DataFrame with new columns for the name of the Exchange.
             Excludes StackOverflow due to size limitations and meta.
    """

    # Extract valid stacks from directory listing
    df['stack_file_name'] = (
        df["Name"].str
        .extract( r"(?P<stack_name>\w*\.stackexchange\.com.7z)"))

    # Filter to valid stacks
    df = (
        df[df['stack_file_name']
        .map(lambda x: isinstance(x, str) and not "meta" in x)]
        .copy())

    # Create nice column
    df['stack_name'] = (
        df['stack_file_name']
        .map(lambda x: x.split(".")[0]))

    df.reset_index(inplace=True,drop=True)

    return df


def download_archive(
    url: str, dest: str, output_widget: iw.Output, **kwargs) -> str:
    """
    Function to download a Stack Exchange archive. It takes in an ipyoutput widget in order to use to update download progress.

    Returns:
    Path: A path object pointing to the downloaded file
    """

    if isinstance(dest, str):
        dest = Path(dest)

    if not dest.exists():
        with output_widget: 
            display(Markdown(f"Please create a folder at the location:\n {download_folder}\n\nNote: You can use relative paths with `..` to mean one folder up. So if you are in a parallel folder to your data folder, `../data` (or `..\\data` on windows) will be what you need. Otherwise, you can always write the full path name such as:\n{download_folder.resolve()} ."))
        return False
    else:
        DOWNLOAD_FOLDER = dest.resolve()

    filename = url.split("/")[-1]
    dest_file = dest / filename

    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))

    if total <= 0: 
        with output_widget:
            print('The file size was 0. Is the URL correct?')
            return False

    with output_widget: 
        with open(dest_file, 'wb') as file, tqdm(
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
        ) as bar:
            for data in resp.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)

    if 'extract' in kwargs and kwargs['extract']:
        if 'keep7z' in kwargs:
            archive_folder = extract_archive(
                dest_file,keep7z=kwargs['keep7z']) 
        return archive_folder
    else:
        return dest_file


def extract_archive(archive_path: Path, **kwargs) -> Path:
    """
    Function to extract a StackExchange archive from 7z.
    The archive is placed in a folder of the same name. 

    Parameters:
    keep7z (Boolean): Keep the original 7z file. It is deleted by default.

    Returns:
    str: the folder path containing the XML files.
    """

    if isinstance(archive_path, str):
        archive_path = Path(archive_path)

    file_name = archive_path.name
    folder_name = ".".join(archive_path.name.split(".")[:-1])
    archive_folder = archive_path.parent / folder_name
    archive_folder.mkdir(exist_ok=True)

    with py7zr.SevenZipFile(archive_path, 'r') as archive:
        archive.extractall(archive_folder)

    if 'keep7z' in kwargs and kwargs['keep7z']:
        pass
    else:
        try:
            os.remove(archive_path)
        except:
            print("The original 7z could not be deleted")

    return archive_folder

from collections.abc import Sequence 

def _seq_but_not_str(obj):
    return isinstance(obj, Sequence) and not isinstance(obj, (str, bytes, bytearray))


def convertXMLtoDataFrame(xml_path: Path, cols = False) -> pd.DataFrame:
    """
    Parser to process an XML file into a DataFrame. Presently assumes it is
    a StackExchange file with the structure of Posts.xml

    Returns:
    pd.DataFrame: Unprocessed DataFrame directory from XML data.
    """

    if isinstance(xml_path, str):
        xml_path = Path(xml_path)

    assert xml_path.exists(), "The file path is not valid"
    
    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Extract the data from the XML elements

    if cols: 
        rows = [{var: row.get(var) for var in cols} 
                 for row in root.findall('row')]
    else: 
        rows = [{attr: row.get(attr) for attr in row.attrib} 
                for row in root.findall('row')]

    # Create a DataFrame from the rows
    return pd.DataFrame(rows)

def is_numeric(series):
    try:
        pd.to_numeric(series, errors='raise')
        return True
    except (ValueError, TypeError):
        return False
    except pd.errors.OutOfBoundsDatetime:
        return False

def convertColumns(df: pd.DataFrame, 
                   exclude_id_cols = True) -> pd.DataFrame:
    """
    Formatter function to change column types to more meaningful types than the default `str` that comes from the XML parsing

    Returns: 
    pd.DataFrame: A DataFrame with the typed columns 
    """

    cols = df.columns
    
    if exclude_id_cols:
        cols = [c for c in cols if not c.endswith("Id")]
    
    for col in cols: 
        if col.endswith('Date'):
            df[col] = pd.to_datetime(df[col], errors='coerce')        
        elif is_numeric(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def parseHTML(text: str, col_name: str, **kwargs) -> dict:
    """
    Parser for HTML comment and returns a tuple with comment text (or None) and list of links (or None)

    Returns:
    dict: Dictionary with data of form {"cleantext": str, "links": list}.
    """

    ret_dict = {}

    #It gets moody because some comments are very terse.
    with warnings.catch_warnings():
        warnings.simplefilter("ignore") 
        if type(text) == float:
            if 'bodytext' in kwargs and kwargs['bodytext']: 
                ret_dict[f'{col_name}Text'] = pd.NA
            if 'links' in kwargs and kwargs['links']: 
                ret_dict[f'{col_name}URLs'] = pd.NA
        else:
            try: 
                soup = bs4.BeautifulSoup(text, 'lxml')
                if 'bodytext' in kwargs and kwargs['bodytext']:
                    ret_dict[f'{col_name}Text'] = soup.text.replace("\n"," ")

                if 'links' in kwargs and kwargs['links']:
                    ret_dict[f'{col_name}URLs'] = [x['href'] for x in soup.find_all('a')
                        if 'href' in x.attrs and "://" in x.get('href')]

            except Exception as e:
                print(type(e), e)

    return ret_dict

def cleanBody(body_series: pd.Series) -> pd.DataFrame:
    """
    Worker that manages the pooled processing of many Body rows to clean.
    """
    col_name = body_series.name

    with multiprocessing.Pool() as pool:
        # Use apply to process each row in parallel
        results = body_series.map(
            lambda row: parseHTML(
                row,bodytext=True,links=True,col_name=col_name))

    return pd.DataFrame(
        results.tolist(), columns=[f'{col_name}Text', f'{col_name}URLs'])

def splitTags(text: str) -> list:
    """
    Formatter that parses text of form `<tag1><tag2><...>` to return a list of `['tag1', 'tag2',...]`

    Returns:
    pd.DataFrame: A two-column DataFrame with the same ID as original for merging.
    """

    if type(text) != str or len(text) == 0:
        return []
    else:
        return text[1:-1].split("><")

def cleanTags(tag_text_series: pd.Series) -> pd.DataFrame:
    """
    Worker that manages the pooled processing of many Tags rows to clean.
    """

    with multiprocessing.Pool() as pool:
        results = tag_text_series.map(splitTags)
    
    new_df = pd.DataFrame(results)
    new_df.columns = ['TagsList']

    return new_df

def df_to_file(df, xml_path, file_type="parquet", **kwargs):

    msg = ""
    
    if file_type == "parquet": 
        import pyarrow.parquet as pq
        import pyarrow as pa
        
        out_path = xml_path.parent / f"{xml_path.stem}.{file_type}"
        
        pq.write_table(pa.Table.from_pandas(df),out_path)
        msg += f"The file is available here: <br> `{str(out_path)}`"
        msg +=  "<br> You can load this file and call `df` with <br> `df = "
        msg += f" pq.read_table('{str(out_path)}').to_pandas()`<br>"

    
    elif file_type == "feather": 
        out_path = xml_path.parent / f"{xml_path.stem}.{file_type}"
        df.to_feather(out_path)
        msg += f"The file is available here: <br> `{str(out_path)}`" 
        msg += f"<br> You can load this file and call `df` "
        msg += f"with <br> `df = pd.read_feather('{str(out_path)}')`"
          
    elif file_type == "pickle":
        import pickle
        out_path = xml_path.parent / f"{xml_path.stem}.{file_type}"
        df.to_pickle(out_path)
        msg += f"The file is available here: <br> `{str(out_path)}`"
        msg += f"<br> You can load this file and call `df` "
        msg += f"with <br> `df = pd.read_pickle('{str(out_path)}')`<br>"

    else:
        pass

    return msg
    
def makeDataFrame(xml_path, file_out="feather", **kwargs):
    """
    Worker that steps through key processing from XML to DataFrame. 

    Parameters:
        extract_body_text (Boolean): 
            Creates new columns for body text, the first has no HTML, 
            the second is a list of links in body text.
        extract_tags (Boolean): Creates a new column for tags as a list 
            rather than a string of the form <tag1><tag2>
        file_out (str): Returns the DataFrame in this format of "pickle", 
            "feather", "csv". Default is feather. CSV is not implemented yet.
    
    Returns:
        str: A file path to the DataFrame that has been saved.
    """

    df = convertXMLtoDataFrame(xml_path)
    df = convertColumns(df)
    concat_cols = [df]

    if 'extract_body_text' in kwargs:
        cols = kwargs['extract_body_text']
        if type(cols) == str: 
            cols = [cols]
            
        if type(cols) == list:
            for col in cols: 
                concat_cols.append(cleanBody(df[col]))
                
    if 'extract_tags' in kwargs and kwargs['extract_tags']:
        concat_cols.append(cleanTags(df['Tags']))
    
    df = pd.concat(
        concat_cols,
        axis = 1)

    return df_to_file(df,xml_path,file_type=file_out)

def display_intro():
    se_intro = iw.Output()
    display(se_intro)


def download_stackexchange_archive():
    """
    Main worker function. Handles the downloading and population of the initial list, displaying options, and reporting outcome to user. 
    """

    se_intro = iw.Output()
    display(se_intro)

    with se_intro:
        display(Markdown("Loading Stack Exchange list..."))

    df = process_archive_table(download_table(URL))

    se_dropdown = iw.Dropdown(
        options= [f"{x[0]}. {x[1]} - {x[2]}" 
                  for x in zip(df.index, df["stack_name"], df["Size"])],
        value=None, 
        description="Archive:")

    se_dropdown.layout.width = '50%'

    se_intro.clear_output()
    with se_intro:
        display(Markdown("Please select a Stack Exchange to process (the book uses 'movies')"))

    se_progress = iw.Output()

    se_download_path = iw.Text(
        description='Folder:',
        value=str(DOWNLOAD_FOLDER))
    
    se_download_path.layout.width = '50%'

    se_download = iw.Button(description="Download", disabled=True)
    
    se_check_keep7z = iw.Checkbox(
        description='Keep downloaded 7z file on disk', value=True) 
    se_check_useXML = iw.Checkbox(
        description='Use existing XML file if found', value=True) 
    se_process_posts = iw.Checkbox(
        description='Process Posts.xml to DataFrame', value=True) 
    se_process_users = iw.Checkbox(
        description='Process Users.xml to DataFrame', value=False) 
    se_check_body_cols = iw.Checkbox(
        description='Insert columns of text stripped of HTML and URLs',
        value=True) 
    se_check_tags_list = iw.Checkbox(
        description='Process Tags column to list of tags',
        value=True) 

    se_file_type = iw.Dropdown(
        options= ["parquet","feather","pickle"],
        value="feather", 
        description="File Type for Export:")

    se_file_type.layout.width = '50%'

    
    def handle_dropdown_change(change):
        '''Internal worker to enable button state once archive selected'''

        selected_option = change.new

        url_index = int(selected_option.split(".")[0])

        se_download.disabled = False
        se_progress.clear_output()

        with se_progress: 
            display(Markdown(
            f"You have selected: `{ df.loc[url_index]['stack_file_name']}`." +
            f"\nThis option is `{df.loc[url_index]['Size']}` in size. " +
             "Click to download."))

    def handle_filetype_change(change):
        se_download.disabled = False
        se_progress.clear_output()
        
    def on_click_worker(_):
        '''Internal worker to manage the process of downloading and processing
        once a button has been pressed'''

        se_download.disabled = True

        with se_progress:

            archive_name = df.loc[
                int(se_dropdown.value.split(".")[0])]["stack_file_name"]
            archive_remote_path = URL + '/' + archive_name
            archive_dest = Path(se_download_path.value).resolve()
            
            dl_posts = se_process_posts.value
            dl_users = se_process_users.value
            
            if se_check_useXML.value:
                
                archive_folder = archive_dest / archive_name.split(".7z")[0]
                
                if se_process_posts.value:
                    if (archive_folder / "Posts.xml").exists():
                        dl_posts = False
                        display(Markdown("Using existing Posts XML file"))
                if se_process_users.value:
                    if (archive_folder / "Users.xml").exists():
                        dl_users = False
                        display(Markdown("Using existing Users XML file"))
                    
            if dl_posts or dl_users:
                archive_folder = download_archive(
                url = archive_remote_path, 
                dest = archive_dest, 
                output_widget = se_progress,
                extract = True,
                keep7z = se_check_keep7z.value)

                display(Markdown("Processing downloaded file..."))

            status = ""
        
            if se_process_posts.value:
                
                body_cols = False
                if se_check_body_cols.value:
                    body_cols="Body"

                status += makeDataFrame(
                    xml_path = archive_folder / ("Posts.xml"),
                    file_out = se_file_type.value,
                    extract_body_text = body_cols,
                    extract_tags = se_check_tags_list.value)

            if se_process_users.value:
                body_cols = False
                if se_check_body_cols.value:
                    body_cols="AboutMe" 
    
                status += makeDataFrame(
                    xml_path = archive_folder / ("Users.xml"),
                    file_out = se_file_type.value,
                    extract_body_text = body_cols)

            display(Markdown(status))

    settings_group = iw.VBox([
        se_check_keep7z,
        se_check_useXML,
        se_check_body_cols,
        se_check_tags_list,
        se_process_posts,
        se_process_users])

    vbox = iw.VBox([
        se_dropdown,
        se_file_type,
        se_download_path, 
        settings_group,
        se_download, 
        se_progress])

    se_dropdown.observe(handle_dropdown_change, names='value')
    se_file_type.observe(handle_filetype_change)

    
    se_download.on_click(on_click_worker)

    display(vbox)

download_stackexchange_archive()

Output()

URLError: <urlopen error [Errno 60] Operation timed out>

## Change log 

## Update V1.2. October 19

Better file management! If the folder with XML files has already been downloaded you can simply run download and it will process the files in there. This is handy if you want to add a Users.xml -> dataframe without redownloading everything. It is also handy if you want to have the files in a different format

New format: parquet, default format remains `feather` as a faster i/o with higher compression and less import overhead. 

## Update V1.1. October 17 

There is a breaking change in this code as I adopt a more consistent naming convention. `ListTags` is now `TagsLists` and `CleanBody` is now `BodyText` and `BodyURLs` in the case of Posts.xml. 

Version 1.1: This version now allows you to download either the `Posts.xml` or `Users.xml` which will be downloaded. The default export version is feather. Anaconda comes with the default packages but if you need to install, it would be `!pip install feather`. Parquet and pickle options are also available for backwards compatibility.

Bug fixes: The export text now is quoted. The column types now enforce Id = Str and do not make int data as floats. The clean HTML code now also removes `\n`. Code is a little more refactored in the XML parser.